In [39]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
%autoreload

import os, random, pandas as pd, numpy as np
from sklearn.model_selection import StratifiedKFold
import pickle
import sys
sys.path.append('../..')
import relation_extraction.data.utils as utils
import itertools
# SEED = 1
# np.random.seed(SEED)
# random.seed(SEED)
preprocessing_type = 'original'
RESOURCE_PATH = "/data/medg/misc/geeticka/relation_extraction/i2b2/pre-processed/" + preprocessing_type
def res(path): return os.path.join(RESOURCE_PATH, path)
if not os.path.exists(res('pickled-files/')):
    os.mkdir(res('pickled-files/'))

In [77]:
train_data = open(res('train_{}.txt').format(preprocessing_type))
splitted_data_border1 = utils.split_data_cut_sentence(train_data, border_size=1)

In [78]:
train_data = open(res('train_{}.txt').format(preprocessing_type))
splitted_data_border20 = utils.split_data_cut_sentence(train_data, border_size=20)

In [79]:
train_data = open(res('train_{}.txt').format(preprocessing_type))
splitted_data_border50 = utils.split_data_cut_sentence(train_data, border_size=50)

In [80]:
train_data = open(res('train_{}.txt').format(preprocessing_type))
splitted_data_border_minus1 = utils.split_data_cut_sentence(train_data)

In [81]:
def get_dataframe(splitted_data):
    # now need to figure out how to create the csv file
    sen = splitted_data[0]
    rel = splitted_data[1]
    e1 = splitted_data[2]
    e2 = splitted_data[3]
    data = pd.DataFrame({'sentences': sen, 'relations': rel, 'e1_pos': e1, 'e2_pos': e2}, 
                        columns=['sentences', 'relations', 'e1_pos', 'e2_pos'])
    return data

In [82]:
data_border1 = get_dataframe(splitted_data_border1)
data_border20 = get_dataframe(splitted_data_border20)
data_border50 = get_dataframe(splitted_data_border50)
data_borderminus1 = get_dataframe(splitted_data_border_minus1)

Size of train data is 2117 and size of test data is 3992 sentences. It is difficult to achieve a similar split in this case because train data is half of the test data. Generally, we train on a larger amount of data than we use for development. So I am choosing to do 5 fold cross validation like in the DDI corpus

In [83]:
def pickle_data(data, filename1, filename2, SEED, preprocessing_type):
    np.random.seed(SEED)
    random.seed(SEED)
    print("seed is", SEED)
    N = len(data)
    K = 5

    splitter = StratifiedKFold(n_splits=K, shuffle=True, random_state=SEED)
    fold_indices = [indices for _, indices in splitter.split(
        data[['sentences', 'e1_pos', 'e2_pos']].values,
        data['relations'].values
    )]

    assert len(fold_indices) == K, "Incorrect number of folds."
    assert len(np.concatenate(fold_indices)) == N, "Folds not comprehensive."

    splits = []
    for fold in range(K):
        test_fold  = fold_indices[fold]
        dev_fold   = fold_indices[(fold + 1) % K]

        non_train_start = fold
        non_train_end   = (fold + 2) % K

        if non_train_start < non_train_end:
            train_fold = np.concatenate(fold_indices[non_train_end:] + fold_indices[:non_train_start])
        else: train_fold = np.concatenate(fold_indices[non_train_end:non_train_start])
        joined = np.concatenate((train_fold, dev_fold, test_fold))

        assert len(joined) == N, (
            "Split not comprehensive for fold {fold}:\n"
            "  len(train_fold): {train}\n"
            "  len(dev_fold): {dev}\n"
            "  len(test_fold): {test}\n"
            "  len(joined): {joined}\n"
            "  N: {N}".format(
                N=N, fold=fold, train=len(train_fold), dev=len(dev_fold), test=len(test_fold), joined=len(joined)
            )
        )

        splits.append((
            data.iloc[train_fold],
            data.iloc[dev_fold],
            data.iloc[test_fold],
        ))

    assert len(splits) == K, "Too few splits"

    #below has the dependency info with the directionalities
    #seed_1_{K}-dep-dir-fold.pkl
    with open(res('pickled-files/' + filename1.format(K=K, p=preprocessing_type)), mode='wb') as f: pickle.dump(splits, f)
    with open(res('pickled-files/'+filename2.format(K=K, p=preprocessing_type)), mode='wb') as f: pickle.dump(splits, f, protocol=2)

In [84]:
pickle_data(data_borderminus1, 'seed_5_{K}-fold-border_-1_{p}.pkl', 'seed_5_{K}-fold-border_-1_{p}_py2.pkl', 5, preprocessing_type)
pickle_data(data_border1, 'seed_5_{K}-fold-border_1_{p}.pkl', 'seed_5_{K}-fold-border_1_{p}_py2.pkl', 5, preprocessing_type)
pickle_data(data_border20, 'seed_5_{K}-fold-border_20_{p}.pkl', 'seed_5_{K}-fold-border_20_{p}_py2.pkl', 5, preprocessing_type)
pickle_data(data_border50, 'seed_5_{K}-fold-border_50_{p}.pkl', 'seed_5_{K}-fold-border_50_{p}_py2.pkl', 5, preprocessing_type)

seed is 5
seed is 5
seed is 5
seed is 5
